In [1]:
import numpy
import random
from collections import defaultdict
import urllib
import math
import random
import collections
import string
import csv

In [3]:
def parseLabeledData(path):
    file=open(path, 'r')
    dataList = []
    for line in csv.reader(file):
        if len(line) == 4:
            dataList.append(
                {"asin":line[0], 
                 "question":line[1],
                 "review":line[2],
                 "label":line[3]}
            )     
    return dataList
        

print("Reading labeled data...")
data = parseLabeledData(R"C:\Users\Moi\Downloads\out.csv")
queries = [d['question'] for d in data]
reviews = [d['review'] for d in data]
labels = [d['label'] for d in data]
print("done")

Reading labeled data...
done


In [4]:
def parseAllQueries(path):
    file = open(path, 'r')
    dataList = defaultdict(lambda: [])
    for line in file:
        line = eval(line)
        dataList[line["asin"]].append(line["question"])
      
    return dataList

def parseAllReviews(path):
    file = open(path, 'r')
    dataList = defaultdict(lambda: [])
    for line in file:
        line = eval(line)
        dataList[line["asin"]].append(line["reviewText"])
      
    return dataList

print("Reading all reviews & all questions...")
allReviews = parseAllReviews(R"C:\Users\Moi\Downloads/reviews.json")
allQuestions = parseAllQueries(R"C:\Users\Moi\Downloads/qa.json")

# do we have to remove questions that have no reviews or reviews that have no questions??
docSet = [review for review in allReviews] + [question for question in allQuestions]

print("done")

Reading all reviews & all questions...
done


In [5]:
def findCommonWords():
    allWords = defaultdict(int)
    for r in allReviews.values():
        for review in r:
            exclude = set(string.punctuation)
            review = ''.join(ch for ch in review if ch not in exclude)
            for w in review.lower().split():
                allWords[w] += 1

    for q in allQuestions.values():
        for question in q:
            exclude = set(string.punctuation)
            question = ''.join(ch for ch in question if ch not in exclude)
            for w in question.lower().split():
                allWords[w] += 1
    
    
    return sorted(allWords, key=lambda x: -allWords[x])[:1000]

In [6]:
commonWords = findCommonWords()

In [7]:
# @param a word whose frequency in the document we are calculating
# @param document a string of a review or a question
# @return the frequency of term in document div length of document

def tf(term, document):
    count = collections.defaultdict(int)
    for word in document.split():
        count[word] += 1

    return count[term]/len(document.split())

In [8]:
def idf(term):
    count = 0         
    for doc in docSet:
        if term in doc.lower():
            count += 1
    return math.log(len(docSet)/(count+1))

In [9]:
def wordToIndex(term):
    if term in commonWords:
        return commonWords.index(term)
    else:
        return -1

In [10]:
def feature(review, question):
    review = review.lower()
    review = ''.join([c for c in review if not (c in string.punctuation)])
    
    featReview = [0]*1000
    
    for term in review.split():
        index = wordToIndex(term)
        if index != -1:
            featReview[index] += 1
            
    question = question.lower()
    question = ''.join([c for c in question if not (c in string.punctuation)])
    
    featQuestion = [0]*1000
    
    for term in question.split():
        index = wordToIndex(term)
        if index != -1:
            featQuestion[index] += 1
            
    return [a*b for a,b in zip(featQuestion,featReview)]

In [96]:
def constrain(elem):
    if elem[0] > 0.5: return [1]
    else: return [0]

In [122]:
def linearRegression():
    X = [feature(d["review"], d["question"]) for d in data]
    y = [1 if l == "y" else 0 for l in labels]
    
    keys = list(range(1, len(labels)))
    points = dict(zip(keys, zip(X, y)))
    random.shuffle(keys)
    X = [points[key][0] for key in keys]
    y = [points[key][1] for key in keys]
    
    X_train = X[:len(X)//2]
    y_train = y[:len(y)//2]
    
    X_test = X[len(X)//2:]
    y_test = y[len(y)//2:]
    
    theta,residuals,rank,s = numpy.linalg.lstsq(X_train, y_train, rcond=None)
    
    X_train = numpy.matrix(X_train)
    y_train = numpy.matrix(y_train).T
    theta = numpy.matrix(theta).T
    y_hatTrain = X_train * theta
    
    X_test = numpy.matrix(X_test)
    y_test = numpy.matrix(y_test).T
    y_hatTest = X_test * theta
    
    # Find mean of Training and Test
    y_barTrain = (sum(y_train) / len(y_train))
    y_barTest = (sum(y_test) / len(y_test))
    
    
    # Constrain y_hat
    y_hatTrainTemp = []
    for y_star in y_hatTrain.tolist():
        y_hatTrainTemp.append(constrain(y_star))
    y_hatTrain = numpy.array(y_hatTrainTemp)
    
    y_hatTestTemp = []
    for y_star in y_hatTest.tolist():
        y_hatTestTemp.append(constrain(y_star))
    y_hatTest = numpy.array(y_hatTestTemp)
    
    y_trainTemp = []
    for y_star in y_train.tolist():
        y_trainTemp.append(constrain(y_star))
    y_train = numpy.array(y_trainTemp)
    
    y_testTemp = []
    for y_star in y_test.tolist():
        y_testTemp.append(constrain(y_star))
    y_test = numpy.array(y_testTemp)
    
    # MSE for Training and Test
    #mseTrain = ((y_hatTrain - y_train)**2).mean()
    #print(mseTrain)
    
    #mseTest = ((y_hatTest - y_test)**2).mean()
    #print(mseTest)
    
    #print(y_hatTrain - y_barTrain)
    #print(numpy.square(y_hatTrain - y_barTrain))
    
    sstot = sum(numpy.square(y_train - y_barTrain))
    ssreg = sum(numpy.square(y_hatTrain - y_barTrain))
    ssres = sum(numpy.square(y_train - y_hatTrain))
    r2 = 1 - ssres/sstot
    #print(ssres, ssreg, ssres + ssreg, sstot)
    print(r2)
    
    sstot = sum(numpy.square(y_test - y_barTest))
    ssreg = sum(numpy.square(y_hatTest - y_barTest))
    ssres = sum(numpy.square(y_test - y_hatTest))
    
    #print(ssres, ssreg, ssres + ssreg, sstot)
    r2 = 1 - ssres/sstot
    print(r2)
    
    
    
    
    
    
    
    
    
    
    
linearRegression()
    
    


[[0.83986361]]
[[-0.31605556]]
